<a href="https://colab.research.google.com/github/DREDESIGNN/AT_F-PYTHON/blob/main/AT_Sistema_de_arquivos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AT-Fundamentos de Desenvolvimento Python

##Aluno: André Luiz Costa de Souza

###Sistema de arquivos, emulado em Python

In [17]:
import random

class Arquivo:
    
    def inic(self, nome):
        self._nome = nome
        self._tamanho = random.randrange(4096)
        
    def getNome(self):
        return self._nome
    
    def getTamanho(self):
        return self._tamanho
    
    def str(self):
        return self._nome + '(' + str(self._tamanho)+')'

In [ ]:
import arquivo

class Diretorio:
    
    def inic(self, nome, dir_pai):
        self._nome = nome
        self._arquivos = {}
        self._subdirs = {}
        self._dir_pai = dir_pai
        
    def new(self, nome_arquivo):
        if (self._arquivos.get(nome_arquivo, -1) == -1):
            arq = arquivo.Arquivo(nome_arquivo)
            self._arquivos[nome_arquivo] = arq
            return True
        return False
        
    def remove(self, nome_arquivo):
        if (self._arquivos.get(nome_arquivo, -1) == -1):
            return False
        del self._arquivos[nome_arquivo]
        return True
    
    def ls(self):
        
        for k,v in self._subdirs.items():
            print(v.getNome()+'   <DIR>')
        
        for k,v in self._arquivos.items():
            print(v)
        print(f'{len(self._arquivos)} arquivo(s) encontrado(s)')
        
    def mkdir(self, nome_dir):
        if (self._subdirs.get(nome_dir, -1) == -1):
            subdir = Diretorio(nome_dir, self)
            self._subdirs[nome_dir] = subdir
            return True
        return False
    
    def getNome(self):
        return self._nome
    
    def getPai(self):
        return self._dir_pai
    
    def temFilho(self, subdir):
        if (subdir != '..'):
            if (subdir in self._subdirs):
                return True
            return False
        return True
    
    def getSubDir(self, subdir):
        if (subdir != '..'):
            if (self.temFilho(subdir)):
                return self._subdirs.get(subdir)
            return None
        return self._dir_pai
        


In [15]:
class Shell:
    
    def inic(self, sist_arq):
        self._fs = sist_arq
    
    def comeca(self):
        saida = False
        print('Inicializando o shell...')

        while (not saida):
            cmd = input('> ')
            acao = cmd.split(' ')
    
            if (acao[0] == 'new'):        
                result = self._fs.newFile(acao[1])
                if (result):
                    print('1 arquivo criado')
                else:
                    print('arquivo já existe')
    
            elif (acao[0] == 'mkdir'):        
                result = self._fs.mkdir(acao[1])
                if (result):
                    print('1 diretório criado')
                else:
                    print('diretório já existe')
            
            elif (acao[0] == 'cd'):        
                result = self._fs.cd(acao[1])
                if (not result):
                    print('diretório inexistente')
            
            elif (acao[0] == 'del'):
                result = self._fs.removeFile(acao[1])
                if (result):
                    print('arquivo removido')
                else:
                    print('arquivo inexistente')
    
            elif (acao[0] == 'ls'):
                if (len(acao) == 1):
                    self._fs.ls()
                else:
                    self._fs.ls(acao[1])
                    
            elif (acao[0] == 'pwd'):
                 print(self._fs.pwd())
    
            elif (acao[0] == 'exit'):
                saida = True
                print('Saindo do shell...')
    
            else:
                print('comando inválido')

In [ ]:
import diretorio
import Shell

class SistemaArquivos:
    
    def inic(self):        
        self._dir_raiz = Diretorio("/", None)
        self._dir_atual = self._dir_raiz
        self._shell = Shell(self)
        
    def comeca(self):
        self._shell.start()
        
    def getDir(self, caminho, caminho_relativo):
        
        if (caminho_relativo):
            prox_dir = self._dir_atual
        else:
            prox_dir = self._dir_raiz
        
        for d in caminho:
            if (prox_dir.temFilho(d)):
                prox_dir = prox_dir.getSubDir(d)
            else:
                return None
            
        return prox_dir
    
    def new(self, arq):               
        caminho = arq.split('/')
        
       
        parte_arq = caminho.pop()

        if (arq[0] == '/'):
            caminho_relativo = False
          
            caminho.pop(0)
        else:
            caminho_relativo = True

        local = self.getDir(caminho, caminho_relativo)
        
        if (local.newFile(parte_arq)):
            return True
        else:
            return False
            
    def ls(self, caminho=None):
        
        
        if (caminho == None):
            self._dir_atual.ls()
            return True
        else:
            caminho_real = caminho.split('/')
            
            if (caminho[0] == '/'):
                caminho_relativo = False
                
                caminho_real.pop(0)
            else:
                caminho_relativo = True

            local = self.getDir(caminho_real, caminho_relativo)
            
            if (local != None):            
                local.ls()
                return True
            
            print('Caminho inválido')
            return False
            
    def mkdir(self, nome_dir):
       
        caminho = nome_dir.split('/')
        parte_arq = caminho.pop()
        
        if (nome_dir[0] == '/'):
            caminho_relativo = False
            
            caminho.pop(0)
        else:
            caminho_relativo = True

        local = self.getDir(caminho, caminho_relativo)
                    
        if (local.mkdir(parte_arq)):
            return True
        else:
            return False
        
    def cd(self, nome_dir):          
    
        caminho = nome_dir.split('/')
        
        if (nome_dir[0] == '/'):
            caminho_relativo = False
            caminho.pop(0)
        else:
            caminho_relativo = True
            
        local = self.getDir(caminho, caminho_relativo)

        if (local != None):
            self._dir_atual = local
            return True
        else:
            return False            
          
    def pwd(self):
        
        raiz = False
        atual = self._dir_atual
        result = '/'
        while (not raiz):            
            pai = atual.getPai()
            if (pai == None):
                raiz = True
            else:
                result = '/' + atual.getNome() + result
            atual = pai
        return result
            
    def getDirAtual(self):
        return self._dir_atual
    